# 部署准备

In [ ]:
!pip install --upgrade git+https://github.com/autodeployai/daas-client.git

In [ ]:
import os
os.environ['TF_KERAS'] = '1'

In [ ]:
from daas_client import DaasClient
client = DaasClient('https://daas.autodeploy.ai', 'admin', 'password')

In [ ]:
project = '部署测试'
if not client.project_exists(project):
        client.create_project(project, 'deployment-test', '部署测试项目')
client.set_project(project)

# 默认部署Keras模型

In [ ]:
from sklearn import datasets
import pandas as pd

# Load the iris dataset (classification)
iris = datasets.load_iris()
iris_target_name = 'Species'
iris_feature_names = iris.feature_names
iris_df = pd.DataFrame(iris.data, columns=iris_feature_names)
iris_df[iris_target_name] = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

seed = 123456
X, y = iris_df[iris_feature_names], iris_df[iris_target_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

# Train a multi-class model using Sequential
model = Sequential()
model.add(Dense(10, input_shape=(4,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(Adam(lr=0.04), 'categorical_crossentropy', ['accuracy'])
model.fit(X_train, pd.get_dummies(y_train).values, epochs=100)

In [ ]:
from pprint import pprint

# Publish the built model into DaaS
publish_resp = client.publish(model,
                              name='keras-iris',
                              mining_function='classification',
                              x_test=X_test,
                              y_test=y_test,
                              description='A Keras classification model')
pprint(publish_resp)

In [ ]:
test_resp = client.test(publish_resp['model_name'], 
                        model_version=publish_resp['model_version'])
pprint(test_resp)

In [ ]:
import requests

response = requests.post(test_resp['endpoint_url'],
                         headers={'Authorization': 'Bearer {token}'.format(token=test_resp['access_token'])},
                         json=test_resp['payload'],
                         verify=False)
pprint(response.json())

In [ ]:
deploy_resp = client.deploy(model_name='keras-iris', 
                            deployment_name='keras-iris-svc',
                            model_version=publish_resp['model_version'],
                            replicas=1)
pprint(deploy_resp)

In [ ]:
response = requests.post(deploy_resp['endpoint_url'],
                         headers={'Authorization': 'Bearer {token}'.format(token=deploy_resp['access_token'])},
                         json=deploy_resp['payload'],
                         verify=False)
pprint(response.json())

# 自定义部署Keras模型

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

# use functional API to build cnn layers
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)
# network model in text
model.summary()

# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train the model with input images and labels
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=3,
          batch_size=batch_size)

In [ ]:
from pprint import pprint

# Publish the built model into DaaS
publish_resp = client.publish(model,
                              name='keras-mnist',
                              mining_function='classification',
                              x_test=x_test,
                              y_test=y_test,
                              description='A tf.Keras classification model')
pprint(publish_resp)

## 通过ONNX部署Keras模型

In [ ]:
import onnxmltools
onnx_model = onnxmltools.convert_keras(model, model.name)

In [ ]:
publish_resp = client.publish(onnx_model,
                              name='keras-mnist-onnx',
                              mining_function='classification',
                              x_test=x_test,
                              y_test=y_test,
                              description='A tf.Keras classification model in ONNX')
pprint(publish_resp)